# Connect to Exasol from AzureML

In this Tutorial we will:
 - create blob storage
 - connect to Exasol SaaS
 - load tables in AzureML
 - save tables to blob storage as csv files


## Prerequisites
 (..)

## Why blob storage is necessary
(explanation)
(word on data duplication?)

## AzureML setup


    - create
        - resource group,
        - workspace,
        - launch  azML studio
        - compute instance
        - notebook (this notebook) link your compute
    - in compute find public ip (if load data was run from inside azurML skip this)
    add to exasol saas like local above

"When you create a workspace, an Azure blob container and an Azure file share are automatically registered as datastores to the workspace. They're named workspaceblobstore and workspacefilestore, respectively. ""https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data"
can find in data in azureml and in azure portal storage account in container as "azureml-blobstore-someID"

if you do not want to use default blobstore
   - create blob storage:
    - in azure portal of storage account:
        -> data storage -> containers -> + container -> in menu on left enter name -> create

        while here:     -> navigate security & networking -> access keys : remember access key and storage account name



create datastore in azureML :(this step might be moved to following tutorial or removed)
    - go to assets -> data  on th left -> datastores -> create
    menu opens on right. ender:
        - datastore name
        - datastore type (we use blob storage because the offered SQL databases are not compatible with exasol atm.)
        - subscription if needed
        - your storage account from dropdown ( from step before) pic
        - newly created blob storage/blob container from step above
        - authentication info (key from step before)
            pic




explanation (link to pyexasol)

In [ ]:
!pip install pyexasol

explanation

In [ ]:
import pyexasol
EXASOL_HOST = "your.clusters.exasol.com" # change
EXASOL_PORT = "8563" # change if needed
EXASOL_USER = "integration-team" # change if needed
EXASOL_PASSWORD = "exa_pat_your_password" #change
EXASOL_SCHEMA = "IDA" # change if needed
EXASOL_CONNECTION = "{host}:{port}".format(host=EXASOL_HOST, port=EXASOL_PORT)
exasol = pyexasol.connect(dsn=EXASOL_CONNECTION, user=EXASOL_USER, password=EXASOL_PASSWORD, compression=True)

# check if working
data = exasol.export_to_pandas("SELECT * FROM TABLE IDA.TEST LIMIT 10")
print(data)

explanation

In [ ]:
import pandas as pd
from azure.ai.ml.entities import AccountKeyConfiguration
from azure.storage.blob import ContainerClient

my_storage_account_name = "your_storage_account_name"
my_storage_account_url = "https://{}.blob.core.windows.net/".format(my_storage_account_name)

credentials= AccountKeyConfiguration(
        account_key="your_storage_account_key"
    )

from azure.storage.blob import ContainerClient
container_client = ContainerClient(account_url = my_storage_account_url,
    container_name = "azureml-tutorial",
    credential= credentials.account_key)

explanation
mention small table size in example

In [ ]:
for table in ["TEST", "TRAIN"]:

    data = exasol.export_to_pandas(f"SELECT * FROM TABLE IDA.{table}")
    data_csv = data.to_csv(index_label="idx", encoding = "utf-8") #todo remove id?

    blob_client = container_client.get_blob_client(f'ida/{table}')
    blob_client.upload_blob(data_csv, overwrite=True)

- Success!
- pic of show tables in AzureML storage